In [ ]:
import fasttext.util
from scipy.spatial.distance import cosine
import pandas as pd

In [ ]:
ft = fasttext.load_model('../input/fasttext-english/cc.en.300.bin')

In [ ]:
def sentence_cosine_similarity(case_num, model, sentences, features):
    ss = [model.get_sentence_vector(s) for s in sentences]
    fs = [model.get_sentence_vector(f) for f in features]
    result = list()
    feature_num = 0
    for i, f in zip(features, fs):
        case = list()
        for j, s in zip(sentences, ss):
            sim = 1 - cosine(s, f)
            if sim > 0.15:
                case.append([case_num, "{:03d}".format(feature_num), sim, i, j])
        result.append(case)
        feature_num += 1
            
    return result

In [ ]:
"""
with open('../input/keyword-extraction/rake.csv', 'r') as f:
    rake = [f.readline().strip().split(',')[:-1] for i in range(10)]
"""
with open('../input/keyword-extraction/yake.csv', 'r') as f:
    yake = [f.readline().strip().split(',')[:-1] for i in range(10)]
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
features['feature_text'] = features['feature_text'].apply(lambda s: s.replace('-', ' '))

In [ ]:
results_yake = [sentence_cosine_similarity(i, ft, yake[i], features[features.iloc[:, 1] == i]['feature_text']) for i in range(10)]

In [ ]:
#results_rake = [sentence_cosine_similarity(ft, rake[i], features[features.iloc[:, 1] == i]['feature_text']) for i in range(10)]

In [ ]:
with open("yake_results.csv",'w', encoding='UTF8') as f:
    f.write('case_num,feature_num,sim,feature,evidence\n')
    for i in results_yake:
        for j in i:
            for k in j:
                f.write(','.join(str(elem) for elem in k))
                f.write('\n')